<a href="https://colab.research.google.com/github/XQuyTran/APP-CQ18/blob/main/notebook/Report_13_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Đồ án môn học "Lập trình song song ứng dụng"

# Xác định loại bệnh thực vật bằng phương pháp trích xuất dữ liệu và mô hình XGBoost.

## Giảng viên: Trần Trung Kiên

### Nhóm 3:
- 18120231 - Trần Xuân Quý
- 18120544 - Trương Minh Tài
- 18120462 - Nguyễn Thị Mận

---

#0. Import







---

# 1. Mô tả ứng dụng

- **Xác định loại bệnh thực vật bằng phương pháp trích xuất dữ liệu và mô hình XGBoost**
	- Input: một ảnh RGB (ảnh lá cây) và một mô hình XGBoost đã train
	- Output: `healthy`	`multiple_diseases`	`rust`	`scab`
	- Ý nghĩa của ứng dụng trong thực tế:
		- tránh các chuẩn đoán sai dẫn đến tăng chi phí, hại môi trường khi sự dụng sai thuốc, hóa chất cho cây trồng
		- chuẩn đoán hiện tại dựa trên con người: tốn thời gian, tốn kém và không mang lại hiệu quả 


![](https://drive.google.com/uc?export=view&id=1H2Nutu74kHXiED9K22Dao5f4mJVVfTyC)









- Lý do cần tăng tốc ứng dụng:
	- Có nhiều ý nghĩa thực tế, có thể áp dụng cho đa dạng loại thực vật; riêng trích xuất dữ liệu có thể áp dụng cho các bài toán dạng ảnh.
	- Ứng dụng này sẽ chạy chậm nếu cài đặt tuần tự
	- Mô hình XGBoost có thể song song hóa và tối ưu hóa.
	





- Khả năng song song hóa
	- Các bước tiền xử lý có thể song song hóa.
	- Các cây độc lập với nhau

---

# 2. Cài đặt tuần tự

## 2.1. Phân tích
Các bước được thực hiện cài đặt tuần tự:
- Chuyển ảnh màu sang ảnh xám.
- Đạo hàm ảnh bằng phương pháp candy.
- Nén ảnh với threshold và liên kết các ô xung quanh để lấp khuyết ảnh.
- Trích xuất color histogram và hue moments
- Lấy trọng số từ mô hình XGBoost và thực hiện tính toán

## 2.2. Thiết kế


####**2.2.1**. **Trích xuất dữ liệu**






> - Sử dụng đạo hàm ảnh bằng phương pháp candy 
    - Giải thuật phát hiện cạnh Canny gồm 4 bước chính sau:
      - **Giảm nhiễu:** làm mờ ảnh, giảm nhiễu dùng bộ lọc Gaussian kích thước `5x5`. Kích thước `5x5` thường hoạt động tốt cho giải thuật Canny.
      - **Tính Gradient và hướng gradient:** dùng bộ lọc Sobel X và Sobel Y (`3x3`) để tính được ảnh đạo hàm Gx và Gy.  Sau đó, tiếp tục tính ảnh Gradient và góc của Gradient theo công thức. Ảnh đạo hàm Gx và Gy là ma trận, thì kết quả tính ảnh đạo hàm Edge Gradient cũng là một ma trận, mỗi pixel trên ma trận này thể hiện độ lớn của biến đổi mức sáng ở vị trí tương ứng trên ảnh gốc. Tương tự, ma trận Angle cũng có cùng kích thước, mỗi pixel trên Angle thể hiện góc, hay nói cách khác là hướng của cạnh. \
      ![](https://firebasestorage.googleapis.com/v0/b/minh-nguyen-blog.appspot.com/o/post-images%2Fopencv%2Fgradient_and_angle.jpg?alt=media&token=4962545e-34bf-4fa5-a9be-f9b6e58ce0a9)
      - **Non-maximum Suppression (viết tắt NMS):** loại bỏ các pixel ở vị trí không phải cực đại toàn cục. Ở bước này, dùng một filter `3x3` lần lượt chạy qua các pixel trên ảnh gradient. Trong quá trình lọc, xem xét xem độ lớn gradient của pixel trung tâm có phải là cực đại (lớn nhất trong cục bộ - local maximum) so với các gradient ở các pixel xung quanh. Nếu là cực đại, ghi nhận sẽ giữ pixel đó lại. Còn nếu pixel tại đó không phải là cực đại lân cận, set độ lớn gradient của nó về zero. Chỉ so sánh pixel trung tâm với 2 pixel lân cận theo hướng gradient. \
       ![](https://firebasestorage.googleapis.com/v0/b/minh-nguyen-blog.appspot.com/o/post-images%2Fopencv%2Fnms.jpg?alt=media&token=48e544e7-a7a1-49f6-8f5b-580aea1c8f44)
      - **Lọc ngưỡng:** xét các pixel dương trên mặt nạ nhị phân kết quả của bước trước. Nếu giá trị gradient vượt ngưỡng max_val thì pixel đó chắc chắn là cạnh. Các pixel có độ lớn gradient nhỏ hơn ngưỡng min_val sẽ bị loại bỏ. Còn các pixel nằm trong khoảng 2 ngưỡng trên sẽ được xem xét rằng nó có nằm liên kề với những pixel được cho là "chắc chắn là cạnh" hay không. Nếu liền kề thì giữ, còn không liền kề bất cứ pixel cạnh nào thì loại. Sau bước này có thể áp dụng thêm bước hậu xử lý loại bỏ nhiễu (tức những pixel cạnh rời rạc hay cạnh ngắn). \
      ![](https://firebasestorage.googleapis.com/v0/b/minh-nguyen-blog.appspot.com/o/post-images%2Fopencv%2Fhysteresis.jpg?alt=media&token=ae128d6f-a4fb-4fa7-ab31-d7a4d1a36d6a)
    - Ảnh sau khi dùng phương pháp Candy \
    ![](https://drive.google.com/uc?export=view&id=1zhOlvtPV0SwMe-TwTsLLqF7O2MDzUp7J)
> - Sử dụng phương pháp nén ảnh với threshold và liên kết các ô xung quanh để lấp khuyết ảnh
    - Hàm dùng để nén ảnh với threshold là ratio sẽ trả ra kết quả là ảnh với block tương ứng với zip x zip và nếu block nào không đủ pixel yêu cầu thì là 0 còn ngược lại thì là 1
    - Hàm dùng để liên kết các ô xung quanh để lấp khuyết sẽ trả ra kết quả là một ma trận mask
    - Ảnh sau khi dùng phương pháp nén ảnh với threshold \
     ![](https://drive.google.com/uc?export=view&id=1rtak3-SXn5qWvKh5eh3EeWGzlezTVbjC)
    - Ảnh sau khi được lấp đầy \
    ![](https://drive.google.com/uc?export=view&id=1zti7WT-tMo7zJy2mqYubxp4aXd-VmiG0)
    - Ảnh sau khi được loại bỏ bớt background \
    ![](https://drive.google.com/uc?export=view&id=1d5edPcyrYHXEzXI4tE3pmG-r86Mi5ATR)
> - Trích xuất color histogram và hue monents
> - Trích xuất dữ liệu từ dạng ảnh sang dạng số


####**2.2.2**. **Dự đoán từ dữ liệu đã được trích xuất và mô hình XGBoost đã train**


Kiến trúc tổng quát của **XGBoost** ![](https://www.researchgate.net/profile/Li-Mingtao-2/publication/335483097/figure/fig3/AS:934217085100032@1599746118459/A-general-architecture-of-XGBoost.ppm) 

**Mô hình XGBoost:** học từ dữ liệu mà không có mô hình cụ thể, được thực hiện bằng cách học không giám sát. **XGBoost** là một thư viện tăng cường gradient để đào tạo GPU, tính toán phân tán và song song, cải thiện đáng kể hiệu suất mô hình. **XGBoost** có độ chính xác cao, thích hợp với tất cả các loại dữ liệu và vấn đề. **XGBoost** là viết tắt của Extreme Gradient Boosting.
Một số tính năng quan trọng của **XGBoost:** 
> - ***Parallelization:*** mô hình được triển khai với nhiều core CPU.
> - ***Regularization:*** mô hình bao gồm các regularization penalty khác nhau để tránh bị overfitting.
> - ***Non-linearity:*** mô hình có thể phát hiện và học từ các mẫu dữ liệu phi tuyến tính. 
> - ***Cross-validation:*** Built-in và comes-out-of-the-box.
> - ***Scalability:*** mô hình có thể chạy phân tán nhờ các máy chủ và cụm phân tán như Hadoop, Spark. Do đó có thể xử lý một lượng lớn dữ liệu.
Một số tham số quan trọng của **XGBRegressor:** 
> - ***Learning_rate:*** được dùng để kiểm soát và điều chỉnh trọng số của internal model estimators. **Learning_rate** phải có giá trị nhỏ để học lâu dài.
> - ***Max_depth:*** cho biết độ sâu của các estimators (tree). Cẩn thận khi điều chỉnh tham số này, vì có thể làm mô hình overfit.
> - ***Alpha:*** một loại regularization penalty để tránh overfitting. 
> - ***Num_estimators:*** số lượng estimators mà mô hình được xây dựng.
> - ***Num_boost_round:*** số lượng giai đoạn tăng cường (boosting). **Num_boost_round** nên được điều chỉnh mỗi khi cập nhật tham số.
 

Các bước triển khai cụ thể:
- Load mô hình và lấy các tham số
- 

## 2.3. Đánh giá

---

# 3. Cài đặt song song

## 3.1. Phân tích

## 3.2. Thiết kế

## 3.3. Đánh giá

---

# 4. Nhìn lại quá trình làm đồ án

**Khó khăn**: nhìn chung cả nhóm đều có nhiều đồ án và deadline, nên thời gian dành cho đồ án chưa thực sự được nhiều để đạt kết quả tốt nhất 
+ Quý: 
+ Tài: 
+ Mận: 


**Học được:**
+ Quý: 
+ Tài: 
+ Mận: 


**Nếu có thêm thời gian**: nhóm sẽ nghiên cứu sâu hơn về ...


---

# 5. Tài liệu tham khảo

Nhập môn học máy: [BÀI TẬP THỰC HÀNH II ](https://drive.google.com/file/d/1U65IfdkizLD0pd1aQKp9xqVRnTj5bejR/view)
